In [2]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [54]:
%pip install aiofiles

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 104.5 kB/s eta 0:00:001m101.9 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [51]:
import os, sys
from dotenv import load_dotenv
rpath = os.path.abspath('/home/user/Documents/10/w11/contract_advisor_rag')

if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [53]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

In [55]:
file_path = rpath + "/data/Raptor Contract.docx"

In [57]:
from docx import Document as DocxDocument
from typing import Iterator
def lazy_load(file_path) -> Iterator[Document]:  # <-- Does not take any arguments
        """A lazy loader that reads a file line by line.

        When you're implementing lazy load methods, you should use a generator
        to yield documents one by one.
        """
        try:
            doc = DocxDocument(file_path)
            line_number = 0
            for para in doc.paragraphs:
                if para.text.strip():  # Skip empty paragraphs
                    yield Document(
                        page_content=para.text,
                        metadata={"line_number": line_number, "source": file_path},
                    )
                    line_number += 1
        except Exception as e:
            print(f"Error reading {file_path}: {str(e)}")

In [59]:
x = lazy_load(file_path)

In [61]:
## Test out the lazy load interface
# for doc in x:
#     print()
#     print(type(doc))
#     print(doc)
x

<generator object lazy_load at 0x74e2e31b5d80>

In [63]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [65]:
documents = text_splitter.split_documents(x)

In [66]:
documents[0]

Document(metadata={'line_number': 0, 'source': '/home/user/Documents/10/w11/contract_advisor_rag/data/Raptor Contract.docx'}, page_content='STOCK PURCHASE AGREEMENT')

In [83]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import chromadb

In [74]:
from dotenv import load_dotenv
load_dotenv()

True

In [78]:
api_key=os.getenv("OPENAI_API_KEY")

In [95]:
persist_directory = rpath + "/data"

In [97]:
embedding_function = OpenAIEmbeddings(  model="text-embedding-ada-002")
persistent_client = chromadb.PersistentClient()
            # Store and load Chroma DB from disk
db = Chroma.from_documents(documents, embedding_function,collection_name="doc", persist_directory=persist_directory,client=persistent_client)

In [101]:
db.similarity_search_with_score("Under what circumstances and to what extent the Sellers are responsible for a breach of representations and warranties")

[(Document(metadata={'line_number': 405, 'source': '/home/user/Documents/10/w11/contract_advisor_rag/data/Raptor Contract.docx'}, page_content='INDIVIDUAL REPRESENTATIONS AND WARRANTIES OF THE SELLERS.'),
  0.2554522454738617),
 (Document(metadata={'line_number': 423, 'source': '/home/user/Documents/10/w11/contract_advisor_rag/data/Raptor Contract.docx'}, page_content='with respect to the Contemplated Transactions for which the Sellers could be liable.'),
  0.25967124104499817),
 (Document(metadata={'line_number': 481, 'source': '/home/user/Documents/10/w11/contract_advisor_rag/data/Raptor Contract.docx'}, page_content='Provisions Concerning the Sellers’ Representative.'),
  0.2599860429763794),
 (Document(metadata={'line_number': 415, 'source': '/home/user/Documents/10/w11/contract_advisor_rag/data/Raptor Contract.docx'}, page_content='REPRESENTATIONS AND WARRANTIES OF THE BUYER.'),
  0.26840740442276)]